In [1]:
import os
import pickle

import keras.utils
import numpy as np
import pandas as pd
from PIL import Image
from keras.applications import Xception
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Activation, Flatten, Dense
from keras.layers import (concatenate)
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
from skimage.transform import resize
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


data_dir = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data"

train_image = os.path.join(data_dir, "train_images")
test_image = os.path.join(data_dir, "test_images")

label_column = "AdoptionSpeed"

BATCH_SIZE, test_size = 512, 0.2

height, width = 100, 100

Using TensorFlow backend.


In [2]:
def read_csv_kaggle(path, is_train):
    train = pd.read_csv(path, sep=',')
    pet_ids = train["PetID"]

    selected_columns = ["Type",
                        "Gender",
                        "Color1",
                        "Color2",
                        "Color3",
                        "MaturitySize",
                        "FurLength",
                        "Vaccinated",
                        "Dewormed",
                        "Sterilized",
                        "Health",
                        #"State",
                        "MaturitySize"]
    
    y = train[label_column] if is_train else None
    
    # One-Hot-encode
    X = pd.get_dummies(train[selected_columns], columns=selected_columns)

    # Normalize:
    to_normalize = ["Age", "Fee", "Quantity"]
    for to_norm in to_normalize:
         X[to_norm] = (train[to_norm] - train[to_norm].mean()) / train[to_norm].std()
    
    return X, y, pet_ids

def read_images(image_paths):
    def random_image():
        return np.random.rand(height, width, 3) * 255
    
    def read_image(path):
        img = np.asarray(Image.open(path).convert("RGB"), dtype="int32" )
        return resize(img, (height, width), anti_aliasing=True, mode='constant')    

    return np.array([read_image(path) if os.path.isfile(path) else random_image() for path in tqdm(image_paths)])

In [3]:
X, y, pet_ids = read_csv_kaggle(os.path.join(data_dir, "train.csv"), True)

In [4]:
f_im_name = "images.binary"

if not os.path.isfile(f_im_name):
    images = read_images([os.path.join(train_image, pet_id + "-1.jpg") for pet_id in pet_ids])
    
    # Standardize:
    mean, std = np.mean(images), np.std(images)
    images_meanstd = (images - mean)/std
    
    with open(f_im_name, 'wb') as handle_1, open("mnstd", 'wb') as handle_2:
        pickle.dump(images_meanstd, handle_1, protocol=pickle.HIGHEST_PROTOCOL)        
        pickle.dump((mean, std), handle_2, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f_im_name, 'rb') as handle_1, open("mnstd", 'rb') as handle_2:
        images_meanstd = pickle.load(handle_1)
        temp = pickle.load(handle_2)
        mean, std = temp[0], temp[1]
                
print(images_meanstd.shape)

(14993, 100, 100, 3)


In [5]:
X_train_img, X_test_img, X_train_else, X_test_else, y_train, y_test = train_test_split(images_meanstd, 
                                                                                       X, 
                                                                                       y, 
                                                                                       test_size=test_size,
                                                                                       random_state=1)

In [20]:
# https://stackoverflow.com/questions/49618986/neural-network-in-keras-with-two-different-input-types-images-and-values
# https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/

transfer = Xception(weights='imagenet', include_top=False, input_shape=(height, width, 3))

# Freeze Xception
#for layer in transfer.layers[:-9]:
#    layer.trainable = False

# Inputs
image_input = Input(shape=(height, width, 3))
aux_input = Input(shape=(len(list(X_train_else)),))

# Images:
transfer = transfer(image_input)
transfer = Dense(150, activation='relu')(transfer)
flatten = Flatten()(transfer)

# Aux input:
x = Dense(150, activation='relu')(aux_input)

# Merged:
merge = concatenate([flatten, x])
x = Dense(500)(merge)
h = Dense(5)(x)

# Predictions:
predictions = Activation('softmax')(h)

model = Model(inputs=[image_input, aux_input], outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit([X_train_img, X_train_else], 
                    keras.utils.to_categorical(y_train),
                    batch_size=BATCH_SIZE, 
                    epochs=4, 
                    validation_split=0.1,
                    callbacks=[ModelCheckpoint('test_model.h5', save_best_only=True)])

Train on 10794 samples, validate on 1200 samples
Epoch 1/4
10794/10794 [==============================] - ETA: 1:44:55 - loss: 1.7289 - acc: 0.07 - ETA: 1:33:55 - loss: 1.6420 - acc: 0.16 - ETA: 1:27:22 - loss: 1.6176 - acc: 0.20 - ETA: 1:21:40 - loss: 1.5830 - acc: 0.21 - ETA: 1:16:37 - loss: 1.5614 - acc: 0.22 - ETA: 1:09:54 - loss: 1.5415 - acc: 0.24 - ETA: 1:03:57 - loss: 1.5319 - acc: 0.25 - ETA: 59:12 - loss: 1.5211 - acc: 0.2578 - ETA: 55:06 - loss: 1.5102 - acc: 0.26 - ETA: 50:55 - loss: 1.5079 - acc: 0.27 - ETA: 46:45 - loss: 1.5007 - acc: 0.28 - ETA: 42:40 - loss: 1.4935 - acc: 0.29 - ETA: 37:45 - loss: 1.4902 - acc: 0.29 - ETA: 33:02 - loss: 1.4894 - acc: 0.29 - ETA: 28:05 - loss: 1.4883 - acc: 0.29 - ETA: 23:18 - loss: 1.4871 - acc: 0.30 - ETA: 18:34 - loss: 1.4844 - acc: 0.30 - ETA: 13:55 - loss: 1.4800 - acc: 0.30 - ETA: 9:42 - loss: 1.4788 - acc: 0.3083 - ETA: 5:01 - loss: 1.4761 - acc: 0.311 - ETA: 22s - loss: 1.4733 - acc: 0.312 - 5954s 552ms/step - loss: 1.4739 - acc:

In [26]:
train_pred = [np.argmax(pred) for pred in model.predict([X_train_img, X_train_else])]
test_predictions = [np.argmax(pred) for pred in model.predict([X_test_img, X_test_else])]

In [27]:
print("Kappa on train: {}".format(round(cohen_kappa_score(y_train, train_pred, weights="quadratic"), 4)))
print("Accuracy on train: {}".format(round(accuracy_score(y_train, train_pred), 4)))
print("________________")
print("Kappa on test: {}".format(round(cohen_kappa_score(y_test, test_predictions, weights="quadratic"), 4)))
print("Accuracy on test: {}".format(round(accuracy_score(y_test, test_predictions), 4)))

Kappa on train: 0.27
Accuracy on train: 0.3805
________________
Kappa on test: 0.2415
Accuracy on test: 0.3531


In [22]:
test, _, test_pet_ids = read_csv_kaggle(os.path.join(data_dir, "test.csv"), False)
test_images = read_images([os.path.join(test_image, pet_id + "-1.jpg") for pet_id in test_pet_ids])
test_images_std = (test_images - mean)/std


  7%|█████▎                                                                         | 266/3972 [00:08<01:24, 44.02it/s]


 14%|███████████▍                                                                   | 575/3972 [00:17<02:08, 26.36it/s]


 21%|████████████████▍                                                              | 828/3972 [00:26<01:20, 39.26it/s]


 29%|██████████████████████▎                                                       | 1137/3972 [00:34<01:18, 36.30it/s]


 35%|███████████████████████████▋                                                  | 1409/3972 [00:43<01:41, 25.27it/s]


...

 78%|████████████████████████████████████████████████████████████▌                 | 3083/3972 [01:37<00:26, 33.93it/s]


 85%|██████████████████████████████████████████████████████████████████▏           | 3373/3972 [01:46<00:23, 25.24it/s]


 92%|███████████████████████████████████████████████████████████████████████▋      | 3653/3972 [01:55<00:09, 32.14it/s]


 99%|█████████████████████████████████████████████████████████████████████████████▎| 3936/3972 [02:04<00:00, 42.03it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 3972/3972 [02:05<00:00, 31.00it/s]

In [23]:
loaded = load_model('test_model.h5')
test_pred = loaded.predict([test_images_std, test])

In [24]:
test_pred_label = [np.argmax(pred) for pred in test_pred]

In [25]:
pd.DataFrame({'PetID': test_pet_ids, 'AdoptionSpeed': test_pred_label}).to_csv('submission4.csv', index=False)